In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import f1_score, classification_report
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
import gc
import matplotlib.pyplot as plt
import seaborn as sns

# 设置样式
sns.set(style='whitegrid')
pd.set_option('display.max_columns', None)

print("库导入完成。")

库导入完成。


In [2]:
# 1. 加载数据
print("加载 train_with_features.csv...")
try:
    df_train = pd.read_csv('train_with_features.csv', low_memory=False)
except FileNotFoundError:
    print("错误：请确保 train_with_features.csv 存在！")
    raise
    # 2. 恢复数据类型
print("恢复数据类型...")
df_train['t'] = pd.to_datetime(df_train['t'])
df_train['ticker_id'] = df_train['ticker_id'].astype('category')

# 3. 目标编码 (5类)
target_map = {'HH': 0, 'HL': 1, 'LH': 2, 'LL': 3}
df_train['class_label_encoded'] = df_train['class_label'].map(target_map).fillna(4).astype(int)

print("数据加载完成。")

加载 train_with_features.csv...
恢复数据类型...
数据加载完成。


In [3]:
# 1. 定义特征
TARGET = 'class_label_encoded'
METADATA_COLS = ['class_label', 'class_label_encoded', 't', 'id', 'train_id']
features_to_drop = [col for col in METADATA_COLS if col in df_train.columns]
FEATURES = df_train.columns.drop(features_to_drop).tolist()

# 2. 时间序列分割 (80/20)
print("按时间分割数据...")
df_train = df_train.sort_values(by=['t', 'ticker_id']).reset_index(drop=True)
split_index = int(len(df_train) * 0.8)

train_data = df_train.iloc[:split_index]
val_data = df_train.iloc[split_index:]

# -------------------------------------------------------
# 💡 【关键修改】 添加 .copy()
# 这告诉 Pandas：我们要建立一个全新的独立对象，
# 之后无论怎么修改 X_train，都与原始的 df_train 无关。
# -------------------------------------------------------
X_train = train_data[FEATURES].copy()
y_train = train_data[TARGET].copy()
X_val = val_data[FEATURES].copy()
y_val = val_data[TARGET].copy()

# 3. 修复数据类型 (关键步骤！)
print("修复数值列类型并填充 NaN...")
numeric_features = [col for col in FEATURES if col != 'ticker_id']

# 转换类型
# (使用 .copy() 后，这里的警告应该会消失)
for col in numeric_features:
    X_train[col] = pd.to_numeric(X_train[col], errors='coerce', downcast='float')
    X_val[col] = pd.to_numeric(X_val[col], errors='coerce', downcast='float')

# 填充 NaN (仅数值列)
X_train[numeric_features] = X_train[numeric_features].fillna(-999)
X_val[numeric_features] = X_val[numeric_features].fillna(-999)

print(f"训练集形状: {X_train.shape}")
print(f"验证集形状: {X_val.shape}")

# 清理内存
del df_train, train_data, val_data
gc.collect()

按时间分割数据...
修复数值列类型并填充 NaN...
训练集形状: (1545, 68732)
验证集形状: (387, 68732)


0

In [4]:
# ====================================================# 配置集成参数# ====================================================
N_MODELS = 10           # 训练 10 个不同的模型 (Batch 数量)
NONE_RATIO = 3          # 噪声:信号 = 3:1 (比 V4 的 1:1 温和，比 V5 的 10:1 激进)
# 1. 计算稀有类数量
counts = Counter(y_train)
n_minority = counts[0] + counts[1] + counts[2] + counts[3]
n_none_target = int(n_minority * NONE_RATIO)

# 确保不超过总数
if n_none_target > counts[4]:
    n_none_target = counts[4]

print(f"--- Bagging 策略 ---")
print(f"稀有信号总数: {n_minority}")
print(f"每个 Batch 中 'None' 的数量: {n_none_target} (比例 1:{NONE_RATIO})")
print(f"每个 Batch 总大小: ~{n_minority + n_none_target}")
print(f"将训练 {N_MODELS} 个模型进行集成...")

# 2. 训练循环
models = []

for i in range(N_MODELS):
    print(f"\n>>> 正在训练模型 {i+1}/{N_MODELS} ...")
    
    # A. 创建 Batch (欠采样)
    # 关键：random_state=i，确保每个模型看到的 'None' 样本是不同的！
    sampling_strategy = {
        0: counts[0], 1: counts[1], 2: counts[2], 3: counts[3], # 保留所有信号
        4: n_none_target  # 采样部分噪声
    }
    rus = RandomUnderSampler(sampling_strategy=sampling_strategy, random_state=i)
    X_batch, y_batch = rus.fit_resample(X_train, y_train)
    
    # B. 定义 LGBM 模型
    # 既然数据量小，我们增加一点正则化防止过拟合
    clf = lgb.LGBMClassifier(
        objective='multiclass',
        num_class=5,
        n_estimators=300,      # 因为数据少，不需要 500 棵树
        learning_rate=0.05,
        num_leaves=31,
        random_state=42,
        n_jobs=-1,
        verbose=-1
    )
    
    # C. 训练
    clf.fit(
        X_batch, y_batch,
        categorical_feature=['ticker_id']
    )
    
    models.append(clf)

print(f"\n成功训练了 {len(models)} 个模型！")

--- Bagging 策略 ---
稀有信号总数: 84
每个 Batch 中 'None' 的数量: 252 (比例 1:3)
每个 Batch 总大小: ~336
将训练 10 个模型进行集成...

>>> 正在训练模型 1/10 ...

>>> 正在训练模型 2/10 ...

>>> 正在训练模型 3/10 ...

>>> 正在训练模型 4/10 ...

>>> 正在训练模型 5/10 ...

>>> 正在训练模型 6/10 ...

>>> 正在训练模型 7/10 ...

>>> 正在训练模型 8/10 ...

>>> 正在训练模型 9/10 ...

>>> 正在训练模型 10/10 ...

成功训练了 10 个模型！


In [5]:
print("开始集成预测 (Soft Voting)...")

# 初始化概率矩阵 (行数 x 5类)
sum_probs = np.zeros((len(X_val), 5))

# 1. 让每个模型都进行预测，并累加概率
for i, model in enumerate(models):
    # predict_proba 返回每个类的概率
    probs = model.predict_proba(X_val)
    sum_probs += probs
    if (i+1) % 2 == 0:
        print(f"已汇总模型 {i+1}/{N_MODELS} 的预测...")

# 2. 取平均
avg_probs = sum_probs / N_MODELS

# 3. 取最大概率对应的类别
val_preds = np.argmax(avg_probs, axis=1)

# 4. 评估
macro_f1_v8 = f1_score(y_val, val_preds, average='macro', zero_division=0)

print("\n========================================")
print(f"📈 Model v8 (Bagging Ensemble) Macro F1: {macro_f1_v8:.4f}")
print("----------------------------------------")
print(f"Model v2 (Best Single): 0.2238")
print(f"Model v4 (Undersample): 0.2250 (Precision low)")
print("========================================")

# 5. 详细报告
target_names = ['HH', 'HL', 'LH', 'LL', 'None']
print("\n详细分类报告 (集成模型):")
print(classification_report(
    y_val, 
    val_preds, 
    target_names=target_names,
    zero_division=0
))

开始集成预测 (Soft Voting)...
已汇总模型 2/10 的预测...
已汇总模型 4/10 的预测...
已汇总模型 6/10 的预测...
已汇总模型 8/10 的预测...
已汇总模型 10/10 的预测...

📈 Model v8 (Bagging Ensemble) Macro F1: 0.1919
----------------------------------------
Model v2 (Best Single): 0.2238
Model v4 (Undersample): 0.2250 (Precision low)

详细分类报告 (集成模型):
              precision    recall  f1-score   support

          HH       0.00      0.00      0.00        10
          HL       0.00      0.00      0.00         9
          LH       0.00      0.00      0.00         4
          LL       0.00      0.00      0.00         5
        None       0.93      0.99      0.96       359

    accuracy                           0.92       387
   macro avg       0.19      0.20      0.19       387
weighted avg       0.86      0.92      0.89       387

